#1.Imports

In [159]:
# Install the Pandas library ydata_profiling
import sys
!{sys.executable} -m pip install -U ydata-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [160]:
import numpy as np
import pandas as pd
import datetime
import json
import plotly.express as px
from sqlalchemy import create_engine, text
from sklearn.impute import KNNImputer, SimpleImputer
from ydata_profiling import ProfileReport
from google.colab import files

# Define the connection string
connection_string = "postgresql://co2-sa-db.postgres.database.azure.com:5432/seattlebeb?user=co2sodapg&password=Greta2023&sslmode=require"

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Create a connection and execute the query
with engine.connect() as conn:
    query = text("SELECT * FROM buildings")
    df = pd.read_sql(query, conn)

df.head()

,osebuildingid,buildingtype,primarypropertytype,zipcode,taxparcelidentificationnumber,councildistrictcode,neighborhood,latitude,longitude,yearbuilt,...,sourceeuiwn_kbtu_sf,siteenergyuse_kbtu,siteenergyusewn_kbtu,steamuse_kbtu,electricity_kbtu,naturalgas_kbtu,defaultdata,compliancestatus,outlier,totalghgemissions
0,1,NonResidential,Hotel,98101.0,0659000030,7,DOWNTOWN,47.61220,-122.33799,1927,...,189.000000,7226362.5,7456910.0,2003882.00,3946027.0,1276453.0,False,Compliant,None,249.98
1,2,NonResidential,Hotel,98101.0,0659000220,7,DOWNTOWN,47.61317,-122.33393,1996,...,179.399994,8387933.0,8664479.0,0.00,3242851.0,5145082.0,False,Compliant,None,295.86
2,3,NonResidential,Hotel,98101.0,0659000475,7,DOWNTOWN,47.61393,-122.33810,1969,...,244.100006,72587024.0,73937112.0,21566554.00,49526664.0,1493800.0,False,Compliant,None,2089.28
3,5,NonResidential,Hotel,98101.0,0659000640,7,DOWNTOWN,47.61412,-122.33664,1926,...,224.000000,6794584.0,6946800.5,2214446.25,2768924.0,1811213.0,False,Compliant,None,286.43
4,8,NonResidential,Hotel,98121.0,0659000970,7,DOWNTOWN,47.61375,-122.34047,1980,...,215.600006,14172606.0,14656503.0,0.00,5368607.0,8803998.0,False,Compliant,None,505.01


#2.Data cleaning

In [161]:
df.min()

<ipython-input-161-c3612c624a3f>:1: FutureWarning: The default value of numeric_only in DataFrame.min is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.min()


osebuildingid                                        1
buildingtype                                    Campus
primarypropertytype                Distribution Center
zipcode                                        98006.0
taxparcelidentificationnumber               0000000000
councildistrictcode                                  1
neighborhood                                   BALLARD
latitude                                      47.49917
longitude                                   -122.41425
yearbuilt                                         1900
numberofbuildings                                  0.0
numberoffloors                                       0
propertygfatotal                                 11285
propertygfaparking                                   0
propertygfabuilding_s                             3636
largestpropertyusetypegfa                       5656.0
secondlargestpropertyusetypegfa                   None
thirdlargestpropertyusetypegfa                     0.0
yearsenerg

In [162]:
# Drop all rows where the energy consumption or CO2 emission columns are negative:
df.drop(index=df[df['sourceeuiwn_kbtu_sf']<0].index, inplace=True)
df.drop(index=df[df['electricity_kbtu']<0].index, inplace=True)
df.drop(index=df[df['totalghgemissions']<=0].index, inplace=True)
df.min()

<ipython-input-162-0e1c56ac19e4>:5: FutureWarning: The default value of numeric_only in DataFrame.min is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.min()


osebuildingid                                        1
buildingtype                                    Campus
primarypropertytype                Distribution Center
zipcode                                        98006.0
taxparcelidentificationnumber               0000000000
councildistrictcode                                  1
neighborhood                                   BALLARD
latitude                                      47.49917
longitude                                   -122.41425
yearbuilt                                         1900
numberofbuildings                                  0.0
numberoffloors                                       0
propertygfatotal                                 11285
propertygfaparking                                   0
propertygfabuilding_s                             3636
largestpropertyusetypegfa                       5656.0
secondlargestpropertyusetypegfa                   None
thirdlargestpropertyusetypegfa                     0.0
yearsenerg

In [163]:
# Fix the number of buildings and floors (assuming that "0" means no additional floor or building, hence "1")
df['numberofbuildings'].replace(0, 1, inplace=True)
df['numberoffloors'].replace(0, 1, inplace=True)

In [164]:
# Fix the values for 'compliancestatus'
df['compliancestatus'].value_counts()

Compliant                       3208
Error - Correct Default Data     112
Non-Compliant                     35
Missing Data                      11
Name: compliancestatus, dtype: int64

In [165]:
df['compliancestatus'].replace('Error - Correct Default Data', np.nan, inplace=True)
df['compliancestatus'].replace('Missing Data', np.nan, inplace=True)
df['compliancestatus'].value_counts()

Compliant        3208
Non-Compliant      35
Name: compliancestatus, dtype: int64

In [166]:
# Fix the values for 'neighborhood'
df['neighborhood'].value_counts()

DOWNTOWN                  571
EAST                      453
MAGNOLIA / QUEEN ANNE     422
GREATER DUWAMISH          375
NORTHEAST                 280
LAKE UNION                250
NORTHWEST                 209
SOUTHWEST                 166
NORTH                     143
BALLARD                   126
CENTRAL                   106
SOUTHEAST                  95
DELRIDGE                   80
North                      41
Central                    26
Northwest                  11
Ballard                     7
Delridge                    4
DELRIDGE NEIGHBORHOODS      1
Name: neighborhood, dtype: int64

In [167]:
df['neighborhood'] = df['neighborhood'].str.upper()
df['neighborhood'] = df['neighborhood'].str.replace('DELRIDGE NEIGHBORHOODS','DELRIDGE')
df['neighborhood'].value_counts()

DOWNTOWN                 571
EAST                     453
MAGNOLIA / QUEEN ANNE    422
GREATER DUWAMISH         375
NORTHEAST                280
LAKE UNION               250
NORTHWEST                220
NORTH                    184
SOUTHWEST                166
BALLARD                  133
CENTRAL                  132
SOUTHEAST                 95
DELRIDGE                  85
Name: neighborhood, dtype: int64

In [168]:
# Remove rows where 'totalghgemissions' is missing
df.dropna(subset=['totalghgemissions'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3357 entries, 0 to 3375
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   osebuildingid                    3357 non-null   int64  
 1   buildingtype                     3357 non-null   object 
 2   primarypropertytype              3357 non-null   object 
 3   zipcode                          3341 non-null   float64
 4   taxparcelidentificationnumber    3357 non-null   object 
 5   councildistrictcode              3357 non-null   int64  
 6   neighborhood                     3357 non-null   object 
 7   latitude                         3357 non-null   float64
 8   longitude                        3357 non-null   float64
 9   yearbuilt                        3357 non-null   int64  
 10  numberofbuildings                3357 non-null   float64
 11  numberoffloors                   3357 non-null   int64  
 12  propertygfatotal    

In [169]:
# Drop all rows where the 'outlier' column is not null
df.drop(index=df[df['outlier'].notna()].index, inplace=True)
df.drop(['outlier'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3325 entries, 0 to 3375
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   osebuildingid                    3325 non-null   int64  
 1   buildingtype                     3325 non-null   object 
 2   primarypropertytype              3325 non-null   object 
 3   zipcode                          3309 non-null   float64
 4   taxparcelidentificationnumber    3325 non-null   object 
 5   councildistrictcode              3325 non-null   int64  
 6   neighborhood                     3325 non-null   object 
 7   latitude                         3325 non-null   float64
 8   longitude                        3325 non-null   float64
 9   yearbuilt                        3325 non-null   int64  
 10  numberofbuildings                3325 non-null   float64
 11  numberoffloors                   3325 non-null   int64  
 12  propertygfatotal    

In [170]:
# Impute 'zipcode' missing values from 'latitude' and 'longitude'
from sklearn.impute import KNNImputer
location = ['zipcode', 'latitude', 'longitude']
dfl = df[location]
dfl = pd.DataFrame(data=KNNImputer(n_neighbors=10).fit_transform(dfl), index=df.index, columns=location)
df = df.drop(columns=location).join(dfl)

In [171]:
df['zipcode'] = df['zipcode'].astype('object')
df['councildistrictcode'] = df['councildistrictcode'].astype('object')
df['numberofbuildings'] = df['numberofbuildings'].astype('int64')
df.dtypes

osebuildingid                        int64
buildingtype                        object
primarypropertytype                 object
taxparcelidentificationnumber       object
councildistrictcode                 object
neighborhood                        object
yearbuilt                            int64
numberofbuildings                    int64
numberoffloors                       int64
propertygfatotal                     int64
propertygfaparking                   int64
propertygfabuilding_s                int64
listofallpropertyusetypes           object
largestpropertyusetype              object
largestpropertyusetypegfa          float64
secondlargestpropertyusetype        object
secondlargestpropertyusetypegfa     object
thirdlargestpropertyusetype         object
thirdlargestpropertyusetypegfa     float64
yearsenergystarcertified           float64
energystarscore                     object
siteeui_kbtu_sf                    float64
siteeuiwn_kbtu_sf                  float64
sourceeui_k

In [172]:
# Derive the buildings age from 'yearbuilt'
current_year = datetime.datetime.now().year
df['age'] = df['yearbuilt'].apply(lambda x: current_year - x)
df.drop(['yearbuilt'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3325 entries, 0 to 3375
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   osebuildingid                    3325 non-null   int64  
 1   buildingtype                     3325 non-null   object 
 2   primarypropertytype              3325 non-null   object 
 3   taxparcelidentificationnumber    3325 non-null   object 
 4   councildistrictcode              3325 non-null   object 
 5   neighborhood                     3325 non-null   object 
 6   numberofbuildings                3325 non-null   int64  
 7   numberoffloors                   3325 non-null   int64  
 8   propertygfatotal                 3325 non-null   int64  
 9   propertygfaparking               3325 non-null   int64  
 10  propertygfabuilding_s            3325 non-null   int64  
 11  listofallpropertyusetypes        3325 non-null   object 
 12  largestpropertyusety

In [173]:
# Fix 'energystarscore'
df['energystarscore'] = df['energystarscore'].replace("NULL", None)
df['energystarscore'] = pd.to_numeric(df['energystarscore'])
df['energystarscore'].describe()

count    2506.000000
mean       67.821229
std        26.703263
min         1.000000
25%        53.000000
50%        75.000000
75%        90.000000
max       100.000000
Name: energystarscore, dtype: float64

According to the brief ("les relevés sont coûteux à obtenir"), the city of Seattle would like to predict CO2 emissions without knowing the energy consumption, so we are only going to keep the type of energy usage for each building :

In [174]:
# Create boolean columns for the types of energy consumption (steam/electricity/gaz)
def energy_usage(cell):
    if cell > 0:
      return True
    else:
      return False
df['steamuse_kbtu'] = df['steamuse_kbtu'].apply(energy_usage)
df['electricity_kbtu'] = df['electricity_kbtu'].apply(energy_usage)
df['naturalgas_kbtu'] = df['naturalgas_kbtu'].apply(energy_usage)
df.rename(columns={'steamuse_kbtu': 'steamuse'}, inplace=True)
df.rename(columns={'electricity_kbtu': 'electricity'}, inplace=True)
df.rename(columns={'naturalgas_kbtu': 'naturalgas'}, inplace=True)
df.drop(['siteeui_kbtu_sf', 'siteeuiwn_kbtu_sf', 'sourceeui_kbtu_sf', 'sourceeuiwn_kbtu_sf', 'siteenergyuse_kbtu', 'siteenergyusewn_kbtu', 'osebuildingid'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3325 entries, 0 to 3375
Data columns (total 29 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   buildingtype                     3325 non-null   object 
 1   primarypropertytype              3325 non-null   object 
 2   taxparcelidentificationnumber    3325 non-null   object 
 3   councildistrictcode              3325 non-null   object 
 4   neighborhood                     3325 non-null   object 
 5   numberofbuildings                3325 non-null   int64  
 6   numberoffloors                   3325 non-null   int64  
 7   propertygfatotal                 3325 non-null   int64  
 8   propertygfaparking               3325 non-null   int64  
 9   propertygfabuilding_s            3325 non-null   int64  
 10  listofallpropertyusetypes        3325 non-null   object 
 11  largestpropertyusetype           3325 non-null   object 
 12  largestpropertyusety

In [175]:
# Drop columns with too many missing values
threshold = 70 # minimum percentage of non-null cells in each column
for column in df.columns:
  if df[column].isnull().sum() / len(df) *100 > (100 - threshold):
      df.drop([column], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3325 entries, 0 to 3375
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   buildingtype                   3325 non-null   object 
 1   primarypropertytype            3325 non-null   object 
 2   taxparcelidentificationnumber  3325 non-null   object 
 3   councildistrictcode            3325 non-null   object 
 4   neighborhood                   3325 non-null   object 
 5   numberofbuildings              3325 non-null   int64  
 6   numberoffloors                 3325 non-null   int64  
 7   propertygfatotal               3325 non-null   int64  
 8   propertygfaparking             3325 non-null   int64  
 9   propertygfabuilding_s          3325 non-null   int64  
 10  listofallpropertyusetypes      3325 non-null   object 
 11  largestpropertyusetype         3325 non-null   object 
 12  largestpropertyusetypegfa      3314 non-null   f

#3.Data Profiling

In [176]:
# Fix for the Google Colab bug "ValueError: Only supported for TrueType fonts" (the report gets generated externally by the profiling.py file)
df.to_csv('co2_eda.csv', index=False)
files.download('co2_eda.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

L'intégralité du rapport est disponible sur https://seabeb.azurewebsites.net/dashboard (ou http://127.0.0.1:8000/dashboard si la web app est arrêtée).

##### Variable dépendante : TotalGHGEmissions
L'immense majorité des immeubles émet moins de 50 tonnes, mais on peut noter la présence de quelques outliers qui polluent énormément.
La rubrique "Interactions" montre que les émissions tendent à croître avec le nombre d'étages, mais aussi avec le score Energy Star, ce qui est beaucoup moins logique : les immeubles les mieux notés devraient au contraire émettre moins de gaz à effet de serre...

La superficie des immeubles (en premier lieu, 'propertygfatotal') est fortement liée au niveau d'émission, comme l'indique la matrice de corrélation.

##### Sélection des features

A choisir entre les trois variables corrélées 'neighborhood', 'councildistrictcode' et 'latitude', on prendra la première pârce que l'implantation des immeubles est souvent liée à la notion de quartier.

La variable 'electricity' peut être négligée parce qu'elle est fortement corrélée au score Energy Star et présente un grand déséquilibre : seulement 5 immeubles ne consomment pas d'électricité.

La variable 'primarypropertytype' sera préférée à 'buildingtype' parce qu'elle est mieux corrélée avec 'totalghgemissions'.

Après le nettoyage des données, il s'avère que la variable 'compliancestatus' est devenue constante sur l'ensemble des lignes restantes, donc on peut la supprimer également. Au final, voici le dataframe qui servira à la modélisation :

In [177]:
# Drop unnecessary features
df.drop(['propertygfabuilding_s', 'largestpropertyusetypegfa', 'councildistrictcode', 'latitude', 'electricity', 'buildingtype', 'defaultdata', 'compliancestatus'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3325 entries, 0 to 3375
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   primarypropertytype            3325 non-null   object 
 1   taxparcelidentificationnumber  3325 non-null   object 
 2   neighborhood                   3325 non-null   object 
 3   numberofbuildings              3325 non-null   int64  
 4   numberoffloors                 3325 non-null   int64  
 5   propertygfatotal               3325 non-null   int64  
 6   propertygfaparking             3325 non-null   int64  
 7   listofallpropertyusetypes      3325 non-null   object 
 8   largestpropertyusetype         3325 non-null   object 
 9   energystarscore                2506 non-null   float64
 10  steamuse                       3325 non-null   bool   
 11  naturalgas                     3325 non-null   bool   
 12  totalghgemissions              3325 non-null   f

In [178]:
# CSV export
df.to_csv('co2_modelling.csv', index=False)
files.download('co2_modelling.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#4.Recommendations

In [182]:
fig = px.bar(df, x="neighborhood", y="totalghgemissions", title="Aménagement des espaces verts", color_discrete_sequence=['green'])
fig.show()
fig.write_html('neighborhood.html')
files.download('neighborhood.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

En supplément du centre-ville, il serait souhaitable de privilégier la partie est de la ville (Northeast+East).

In [183]:
fig = px.scatter(df, x='age', y='totalghgemissions', size='totalghgemissions', color_discrete_sequence=['red'], title="Adaptation des politiques de réhabilitation des immeubles")
fig.show()
fig.write_html('age.html')
files.download('age.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Outre les immeubles très anciens, ceux qui ont entre 20 et 40 ans pourraient être intégrés dans les politiques de subventionnement des travaux.

In [184]:
fig = px.scatter(df, x='energystarscore', y='totalghgemissions', size='totalghgemissions', color_discrete_sequence=['blue'], title="Nécessité d'un nouveau label Energy Star")
fig.show()
fig.write_html('estar.html')
files.download('estar.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

L'absence de corrélation entre les émissions de CO2 et le score Energy Star semble justifier la création d'un nouveau label, tel qu'il a été envisagé dans ce papier : https://www.researchgate.net/publication/342831494_EnergyStar_Towards_more_accurate_and_explanatory_building_energy_benchmarking